# Process of converting FHS dataset to UDS format

Note: Please Download FHS datasets from (<a href="https://www.framinghamheartstudy.org/fhs-for-researchers/data-available-overview/">www.framinghamheartstudy.org/fhs-for-researchers/data-available-overview/</a>) prior to running this script.

In [ ]:
import pandas as pd
import numpy as np
import sys
import csv

csv.field_size_limit(sys.maxsize)

## Load the data

In [ ]:
cu0 = pd.read_csv('Generation-1/curated_bap_0_0919_nolabel.csv')
cu0 = cu0[~cu0['demrv046'].isna()].dropna(subset=['normal_date', 'impairment_date', 'mild_date', 'moderate_date', 'severe_date'], how='all')
cu0.columns = cu0.columns.str.lower()

cu17 = pd.read_csv('Generation-2/curated_bap_17_0712_nolabel.csv')
cu17 = cu17[~cu17['demrv046'].isna()].dropna(subset=['normal_date', 'impairment_date', 'mild_date', 'moderate_date', 'severe_date'], how='all')
cu17.columns = cu17.columns.str.lower()

mri = pd.read_csv('FHS_MRI_All.csv')
mri.columns = mri.columns.str.lower()

# Maximum difference between different test visits
diff_months = 6

In [6]:
# convert dates to pandas datetime object
for col in cu0.columns:
    try:
        # Try converting each column to datetime
        cu0[col] = pd.to_datetime(cu0[col], format='%m/%d/%Y', errors='raise')
    except ValueError:
        # If conversion fails, move to the next column
        continue
    
    
for col in cu17.columns:
    try:
        # Try converting each column to datetime
        cu17[col] = pd.to_datetime(cu17[col], format='%m/%d/%Y', errors='raise')
    except ValueError:
        # If conversion fails, move to the next column
        continue

In [8]:
# pivot on mri dates and mri locations
def convert_date(date_int):
    date_str = str(date_int)
    if len(date_str) != 8:
        raise ValueError("Date must be in yyyymmdd format")
    year = date_str[0:4]
    month = date_str[4:6]
    day = date_str[6:8]
    return f"{month}/{day}/{year}"

mri['date'] = mri['date'].apply(convert_date)

mri['date_rank'] = mri.groupby('id').cumcount() + 1

# Pivot the DataFrame for 'date'
mri_pivot_date = mri.pivot_table(index=['id', 'idtype'], columns='date_rank', values='date', aggfunc='first').reset_index()
mri_pivot_date.columns = ['id', 'idtype'] + [f'mri_date{i}' for i in range(1, len(mri_pivot_date.columns) - 1)]

# Pivot the DataFrame for 'dst'
mri_pivot_dst = mri.pivot_table(index=['id', 'idtype'], columns='date_rank', values='dst', aggfunc='first').reset_index()
mri_pivot_dst.columns = ['id', 'idtype'] + [f'mri_dst{i}' for i in range(1, len(mri_pivot_dst.columns) - 1)]

# Pivot the DataFrame for 'id_date'
mri_pivot_id_date = mri.pivot_table(index=['id', 'idtype'], columns='date_rank', values='id_date', aggfunc='first').reset_index()
mri_pivot_id_date.columns = ['id', 'idtype'] + [f'mri_id_date{i}' for i in range(1, len(mri_pivot_id_date.columns) - 1)]

# Merge the two pivoted DataFrames
mri_combined = pd.merge(mri_pivot_date, mri_pivot_dst, on=['id', 'idtype'])

mri_combined = pd.merge(mri_combined, mri_pivot_id_date, on=['id', 'idtype'])

for col in mri_combined.columns:
    try:
        # Try converting each column to datetime
        mri_combined[col] = pd.to_datetime(mri_combined[col], format='%m/%d/%Y', errors='raise')
    except ValueError:
        # If conversion fails, move to the next column
        continue

In [9]:
# merge mris with data
cu0_merged = pd.merge(cu0, mri_combined, on=['idtype', 'id'], how='left')
cu17_merged = pd.merge(cu17, mri_combined, on=['idtype', 'id'], how='left')

In [15]:
# get date of diagnosis
def get_diagnosis_date(row):
    # print(row['mild_date'])
    if isinstance(row['mild_date'], pd.Timestamp):
        row['diag_date'] = row['mild_date']
    elif isinstance(row['moderate_date'], pd.Timestamp):
        row['diag_date'] = row['moderate_date']
    elif isinstance(row['severe_date'], pd.Timestamp):
        row['diag_date'] = row['severe_date']
    elif isinstance(row['impairment_date'], pd.Timestamp):
        row['diag_date'] = row['impairment_date']
    elif isinstance(row['normal_date'], pd.Timestamp):
        row['diag_date'] = row['normal_date']
    return row

cu0_merged = cu0_merged.apply(get_diagnosis_date, axis=1)
cu17_merged = cu17_merged.apply(get_diagnosis_date, axis=1)


## Get the closest visit information

In [18]:
# extract the visit numbers for different exams (date_core, np, mri) within 6 months of diagnosis date

import re
def extract_number(col_name):
    """Extract the trailing number from the column name."""
    match = re.search(r'(\d+)$', col_name)
    return int(match.group()) if match else 0

def select_tests(row):
    cols = row.index
    date_core = sorted([col for col in cols if 'date_core' in col], key=extract_number, reverse=True)
    examdate_np = sorted([col for col in cols if 'examdate_np' in col], key=extract_number, reverse=True)
    mri_date = sorted([col for col in cols if 'mri_date' in col], key=extract_number, reverse=True)
    
    diff = pd.DateOffset(months=diff_months)
    # print((row['diag_date'] - diff))
    
    for col in date_core:
        if (not isinstance(row[col], float)) and (not isinstance(row[col], str)) and (not pd.isna(row[col])) and (row[col] >= (row['diag_date'] - diff) and row[col] <= (row['diag_date'] + diff)):
            # row['_'.join(col.split('_')[:-1]) + '_core'] = row[col]
            row['_'.join(col.split('_')[:-1]) + '_core_no'] = int(re.search(r'(\d+)$', col).group())
            break
        # row['_'.join(col.split('_')[:-1]) + '_core'] = pd.to_datetime(np.NaN)
        row['_'.join(col.split('_')[:-1]) + '_core_no'] = np.NaN
        
    for col in examdate_np:
        if (not isinstance(row[col], float)) and (not isinstance(row[col], str)) and (not pd.isna(row[col]))  and (row[col] >= (row['diag_date'] - diff) and row[col] <= (row['diag_date'] + diff)):
            # row['_'.join(col.split('_')[:-1]) + '_np'] = row[col]
            row['_'.join(col.split('_')[:-1]) + '_np_no'] = int(re.search(r'(\d+)$', col).group())
            break
        # row['_'.join(col.split('_')[:-1]) + '_np'] = pd.to_datetime(np.NaN)
        row['_'.join(col.split('_')[:-1]) + '_np_no'] = np.NaN
    
    for col in mri_date:
        if (not isinstance(row[col], float)) and (not isinstance(row[col], str)) and (not pd.isna(row[col]))  and (row[col] >= (row['diag_date'] - diff) and row[col] <= (row['diag_date'] + diff)):
            # row['_'.join(col.split('_')[:-1]) + '_date'] = row[col]
            row['_'.join(col.split('_')[:-1]) + '_date_no'] = int(re.search(r'(\d+)$', col).group())
            break
        # row['_'.join(col.split('_')[:-1]) + '_date'] = pd.to_datetime(np.NaN)
        row['_'.join(col.split('_')[:-1]) + '_date_no'] = np.NaN
        
    return row
    
cu0_merged_ = cu0_merged.apply(select_tests, axis=1)
cu17_merged_ = cu17_merged.apply(select_tests, axis=1) 

In [19]:
# extract all test values for the selected visit numbers
def get_correct_columns(row):
    cols = row.index
    # core_values = set(['_'.join(col.split('_')[:-1]) + '_core' for col in cols if '_core' in col.lower() and 'date_col' not in col])
    core_values = set()
    for col in cols:
        if 'date_core' in col:
            continue
        if '_core' in col.lower():
            tmp = col.split('_')
            for i, wrd in enumerate(tmp):
                if 'core' in wrd.lower():
                    tmp[i] = 'core'
                    break
            core_values.add('_'.join(tmp))
        
    np_values = set(['_'.join(col.split('_')[:-1]) + '_np' for col in cols if '_np' in col.lower() and 'examdate_np' not in col])
    # print(core_values)
    # print(type(row['date_core_no']), row['date_core_no'])
    if 'date_core_no' in row and not pd.isna(row['date_core_no']):
        date_core_no = int(row['date_core_no'])
        row['date_core'] = row[f'date_core{date_core_no}']
        for col in core_values:
            # print(f'{core_col}{exam_cycle}', index)
            tmp = col.split('_')
            for i, wrd in enumerate(tmp):
                if 'core' in wrd:
                    tmp[i] = f'core{date_core_no}'
            tmp = '_'.join(tmp)
            if tmp in row:
                if col != '_'.join(col.split('_')[:-1]) + '_core':
                    print(col, '_'.join(col.split('_')[:-1]) + '_core')
                row[col] = row[tmp]
            else:
                row[col] = np.NaN
    else:
        if 'date_core' not in row.index:
            row['date_core'] = pd.to_datetime(np.NaN)
        for col in core_values:
            if col not in row.index:
                row[col] = np.NaN
    
    
    if 'examdate_np_no' in row and not pd.isna(row['examdate_np_no']):    
        examdate_np_no = int(row['examdate_np_no'])
        row['examdate_np'] = row[f'examdate_np{examdate_np_no}']
                
        for col in np_values:
            # print(f'{core_col}{exam_cycle}', index)
            if col != '_'.join(col.split('_')[:-1]) + '_np':
                print(col, '_'.join(col.split('_')[:-1]) + '_np')
            if f'{col}{examdate_np_no}' in row:
                row[col] = row[f'{col}{examdate_np_no}']
            else:
                row[col] = np.NaN
    else:
        if 'examdate_np' not in row.index:
            row['examdate_np'] = pd.to_datetime(np.NaN)
        for col in np_values:
            if col not in row.index:
                row[col] = np.NaN
            
    if 'mri_date_no' in row and not pd.isna(row['mri_date_no']):
        mri_date_no = int(row['mri_date_no'])
        row['mri_date'] = row[f'mri_date{mri_date_no}']
        row['mri_dst'] = row[f'mri_dst{mri_date_no}']
        row['mri_id_date'] = row[f'mri_id_date{mri_date_no}']
    else:
        if 'mri_date' not in row.index:
            row['mri_date'] = pd.to_datetime(np.NaN)
        
        if 'mri_dst' not in row.index:
            row['mri_dst'] = np.NaN
            
        if 'mri_id_date' not in row.index:
            row['mri_id_date'] = np.NaN
            
    return row

# cu0_merged_[:100].apply(get_correct_columns, axis=1)

cu0_merged_conv = cu0_merged_.apply(get_correct_columns, axis=1)
cu17_merged_conv = cu17_merged_.apply(get_correct_columns, axis=1)

In [20]:
cu0_merged_conv.to_csv(f'fhs_cu0_merged_conv_{diff_months}months.csv', index=False)
cu17_merged_conv.to_csv(f'fhs_cu17_merged_conv_{diff_months}months.csv', index=False)

In [22]:
def cdr_val(row):
    if isinstance(row['severe_date'], pd.Timestamp):
        return 3.0
    elif isinstance(row['moderate_date'], pd.Timestamp):
        return 2.0
    elif isinstance(row['mild_date'], pd.Timestamp):
        return 1.0
    elif isinstance(row['impairment_date'], pd.Timestamp):
        return 0.5
    elif isinstance(row['normal_date'], pd.Timestamp):
        return 0.0
    return np.NaN

# get final columns and merge the two datasets
intersect = list(set(list(cu0_merged_conv.columns)).intersection(set(list(cu17_merged_conv.columns))))
intersect = [col for col in intersect if 'mri_date' not in col and 'mri_dst' not in col and 'mri_id_date' not in col and '_np' not in col and '_core' not in col]
columns = sorted(intersect) + ['date_core', 'examdate_np', 'mri_date', 'mri_dst', 'mri_id_date', 'date_core_no', 'examdate_np_no', 'mri_date_no'] + list(set(['_'.join(col.split('_')[:-1]) + '_core' for col in cu0_merged_.columns.to_list() if '_core' in col.lower() and 'date_core' not in col])) + list(set(['_'.join(col.split('_')[:-1]) + '_np' for col in cu0_merged_.columns.to_list() if '_np' in col.lower() and 'examdate_np' not in col]))

cu0_final = cu0_merged_conv[columns]
cu17_final = cu17_merged_conv[columns]
# cu0_final['race_code'] = 'W'
combined = pd.concat([cu0_final, cu17_final], axis=0)
combined['cdr'] = combined.apply(cdr_val, axis=1)


In [23]:
combined.to_csv(f'fhs_combined_cu0_cu17_{diff_months}months.csv', index=False)

## Convert to UDS

In [27]:
def hispanic(row):
    if (isinstance(row['race_code'], str)):
        if 'E' in row['race_code']:
            return 0.0
        elif 'H' in row['race_code']:
            return 1.0
        else:
            return np.NaN
    else:
        return np.NaN

def race(row):
    if not isinstance(row['race_code'], str):
        return np.NaN
    codes = set([code for code in list(row['race_code']) if code != 'E' and code != 'H'])
    if len(codes) > 1 or len(codes) == 0:
        return np.NaN
    matches = {'W': 1, 'B': 2, 'N': 3, 'P': 4, 'A': 5, 'O': 50, 'X': np.NaN, 'R': np.NaN}
    return matches[list(codes)[0]]

def convert_variables(df):
    df['MRI'] = df['mri_id_date']
    # df['HISPANIC'] = df.apply(hispanic, axis=1)
    # df['NACCNIHR'] = df.apply(race, axis=1)
    df['HISPANIC'] = 0.0
    df['RACE'] = 1.0
    df['NACCNIHR'] = 1.0
    df['SEX'] = df['sex']
    # df['NACCNIHR'] = df['race_code'].replace({'W': 1, 'B': 2, 'A': 5, 'P': 4, 'N': 3, 'O': 50, 'R': 99}).astype(float)
    
    df['NACCAGE'] = df['age_core'].astype(float)
    df['WEIGHT'] = df['weight_core'].astype(float)
    df['HEIGHT'] = df['height_core'].astype(float)
    df['NACCBMI'] = df['bmi_core'].astype(float)
    df['TOBAC30'] = df['smoking_core'].astype(float)
    df['BPSYS'] = df['sbp_core'].astype(float)
    df['BPDIAS'] = df['dbp_core'].astype(float)
    df['NACCDBMD'] = df['dmrx_core'].astype(float)
    df['NACCLIPL'] = df['liprx_core'].astype(float)
    df['NACCMMSE'] = df['cogscr_core'].astype(float)
    df['HXSTROKE'] = df['prevalent_stroke_core'].replace({0.0: 0, 1.0: 2}).astype(float)
    df['AFIBRILL'] = df['prevalent_af_core'].astype(float)
    
    df['MARISTAT'] = df['marital_np'].replace({1.0: 5, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 9}).astype(float)
    df['HANDED'] = df['handedness_np'].replace({1.0: 2, 2.0: 1, 3.0: 3}).astype(float)
    df['BOSTON'] = df['bnt30_np'].astype(float)
    
    df['CDRGLOB'] = df['cdr'].astype(float)
    
    return df

combined_converted = convert_variables(combined)

In [28]:
combined_converted['NACCNIHR'].value_counts()

1.0    2970
Name: NACCNIHR, dtype: int64

In [29]:
labels = ['AD', 'LBD', 'VD', 'FTD']
for lab in labels + ['NC', 'MCI', 'DE']:
    combined_converted[lab] = np.NaN

def labels_conversion(row):
    # print(type(row['demrv046']), type(row['demrv103']))
    # if not isinstance(row['demrv046'], float):
    #     return row
    if row['demrv046'] == 0 and row['demrv103' ] == 0:
        row['NC'] = 1
        row['MCI'] = 0
        row['DE'] = 0
        for lab in labels:
            row[lab] = 0
    elif row['demrv046'] == 0 and row['demrv103' ] == 9:
        row['NC'] = 0
        row['MCI'] = 1
        row['DE'] = 0
        for lab in labels:
            row[lab] = 0
            
    elif row['demrv046'] == 1:
        row['NC'] = 0
        row['MCI'] = 0
        row['DE'] = 1
        
        if row['demrv103' ]== 1 or row['demrv103' ]== 2:
            row['AD'] = 1
            for lab in labels:
                if lab != 'AD':
                    row[lab] = 0
        elif row['demrv103' ]== 3:
            row['VD'] = 1
            for lab in labels:
                if lab != 'VD':
                    row[lab] = 0
        elif row['demrv103' ]== 4:
            row['AD'] = 1
            row['VD'] = 1
            for lab in labels:
                if lab != 'AD' and lab != 'VD':
                    row[lab] = 0
        elif row['demrv103' ]== 5:
            row['FTD'] = 1
            for lab in labels:
                if lab != 'FTD':
                    row[lab] = 0
        elif row['demrv103' ]== 6:
            row['LBD'] = 1
            for lab in labels:
                if lab != 'LBD':
                    row[lab] = 0
    
    return row

combined_converted = combined_converted.apply(labels_conversion, axis=1)
combined_converted.dropna(subset=['NC', 'MCI', 'DE'] + labels, inplace=True)

In [30]:
final_columns = ['id', 'idtype', 'framid', 'demrv046', 'demrv103', 'normal_date', 'impairment_date', 'mild_date', 'moderate_date', 'severe_date', 'diag_date', 'mri_date', 'mri_dst'] + ['NC', 'MCI', 'DE'] + labels + ['MRI', 'SEX', 'HISPANIC', 'RACE', 'NACCNIHR', 'NACCAGE', 'WEIGHT', 'HEIGHT', 'NACCBMI', 'TOBAC30', 'BPSYS', 'BPDIAS', 'NACCDBMD', 'NACCLIPL', 'NACCMMSE', 'HXSTROKE', 'AFIBRILL', 'MARISTAT', 'HANDED', 'BOSTON', 'CDRGLOB', 'neuropath_avail']

In [31]:
combined_final = combined_converted[final_columns]
combined_final.to_csv(f'fhs_converted_{diff_months}months.csv', index=False)